In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [16]:
import llama_index

In [25]:
from llama_index.core import  VectorStoreIndex,SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [26]:
documents

[Document(id_='4b97573a-35e1-4246-9180-e05f0bca69d9', embedding=None, metadata={'page_label': '1', 'file_name': 'Attention.pdf', 'file_path': 'g:\\LLAMAINDEX\\data\\Attention.pdf', 'file_type': 'application/pdf', 'file_size': 1439172, 'creation_date': '2024-08-17', 'last_modified_date': '2024-08-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoo

In [27]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)

g:\LLAMAINDEX\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 48/48 [00:03<00:00, 15.49it/s]


In [28]:
index

In [30]:
query_engine=index.as_query_engine()

In [31]:
query_engine

In [43]:
response=query_engine.query('what is transformers?')

In [44]:
print(response)

The Transformer is a model architecture that relies entirely on an attention mechanism to establish global dependencies between input and output, eliminating the need for recurrence in sequence transduction tasks. This approach allows for increased parallelization during training and has shown significant improvements in translation quality, outperforming models based on recurrent or convolutional layers.


In [59]:
response=query_engine.query('what is yolo?')

In [61]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

Final Response: YOLO is an acronym that stands for "You Only Look
Once."
______________________________________________________________________
Source Node 1/2
Node ID: 23b14baa-dc42-463e-8c1d-fa4542feeec3
Similarity: 0.7951360163643367
Text: 2, 5 [62] Joseph Redmon and Ali Farhadi. YOLO9000: better,
faster, stronger. In Proceedings of the IEEE/CVF Conference on
Computer Vision and Pattern Recognition (CVPR) , pages 7263–7271,
2017. 2 [63] Joseph Redmon and Ali Farhadi. YOLOv3: An incremental
improvement. arXiv preprint arXiv:1804.02767 , 2018. 1, 2[64] Hamid
Rezatoﬁghi, Nathan Tsoi,...
______________________________________________________________________
Source Node 2/2
Node ID: 37bacd84-d1f1-440b-90bf-5883dea2385b
Similarity: 0.7941252984329317
Text: YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for
real-time object detectors Chien-Yao Wang1, Alexey Bochkovskiy, and
Hong-Yuan Mark Liao1 1Institute of Information Science, Academia
Sinica, Taiwan kinyiu@iis.sinica.edu.tw

In [67]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

In [68]:
# Initialize the VectorIndexRetriever to fetch documents based on vector similarity.
# 'index' is the index where your documents are stored.
# 'similarity_top_k' specifies how many top similar documents to retrieve.
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=4)

In [69]:
# Initialize the RetrieverQueryEngine to handle queries and interact with the retriever.
query_engine = RetrieverQueryEngine(retriever=vector_retriever)

# Initialize the SimilarityPostprocessor to refine the retrieved results.
# 'similarity_cutoff' sets the threshold for considering documents as similar.
# Only documents with a similarity score above this threshold (e.g., 0.80) are included.
similarity_postprocessor = SimilarityPostprocessor(similarity_cutoff=0.80)

In [70]:
query_engine=RetrieverQueryEngine(retriever=vector_retriever,node_postprocessors=[similarity_postprocessor])

In [71]:
response=query_engine.query("What is attention is all yopu need?")

In [75]:
pprint_response(response)
print()
print(response)

Final Response: The attention is all you need is a new simple network
architecture called the Transformer, which is based solely on
attention mechanisms. It does not rely on complex recurrent or
convolutional neural networks like traditional sequence transduction
models. The Transformer model has shown superior quality in tasks such
as machine translation, is more parallelizable, and requires
significantly less training time compared to existing models.

The attention is all you need is a new simple network architecture called the Transformer, which is based solely on attention mechanisms. It does not rely on complex recurrent or convolutional neural networks like traditional sequence transduction models. The Transformer model has shown superior quality in tasks such as machine translation, is more parallelizable, and requires significantly less training time compared to existing models.


Save Index in local And Get Index from local 

In [77]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What are transformers?")
print(response)

Transformers are a model architecture that relies entirely on an attention mechanism to draw global dependencies between input and output. They do not use recurrence and are designed to allow for significantly more parallelization compared to traditional models like recurrent neural networks.
